In [218]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

# Lista Cenas Importantes

Erro mt random a povoar references

Ver parte das sequences --> Precisamos de mais espaço mas o SQL limita isso  Data too long for column 'Translacion_seq' at row 1

Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Genbank`, CONSTRAINT `Genbank-references` FOREIGN KEY (`ID`) REFERENCES `References` (`ID`))

Ligar Base de Dados

In [219]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()


Ficheiro GB 

In [220]:
id_genbank_input = input("ID NCBI")

In [221]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

In [222]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Locus

In [223]:
# Locus 
ID = record.name
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [224]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

NC_069360
153821
DNA circular
PLN
10-JAN-2023


In [225]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

In [226]:
# cur.execute(f"""
#                 INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
#                 VALUES (%s, %s, %s, %s, %s)
#                 """,
#                     (ID, Size, Molecular_type, Genbank_division, Modification_date))

# Tabela Genbank

Usamos


In [227]:
# Genbank
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""
ID_version_seq = record.id
for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [228]:
print(ID_version_seq)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

NC_069360.1
Stenopetalum nutans
NC_069360
Stenopetalum nutans voucher MSB86929 chloroplast, complete genome
RefSeq
...


In [229]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID_version_seq, Organism, Accession, Definition, Keywords, Id_locus)    
                VALUES (%s, %s, %s, %s, %s, %s)
                """,
                        (ID_version_seq, Organism, Accession, Definition, Keywords,ID ))
except:
        print("Erro Povoação Genbank")

In [230]:
# cur.execute(f"""
#                  INSERT INTO Genbank (ID_version_seq, Organism, Accession, Definition, Keywords, Id_locus)    
#                 VALUES (%s, %s, %s, %s, %s, %s)
#                 """,
#                         (ID_version_seq, Organism, Accession, Definition, Keywords, ID ))

# Tabela Sequences  / Parte para Algoritmos

In [231]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [232]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

NC_069360
...
48124
28562
27605
49530


In [233]:
try:

        cur.execute("""
                INSERT INTO Sequences ( Count_A, Count_C, Count_T, Count_G, Sequence, ID_version_seq)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (Count_A, Count_C, Count_T, Count_G, Sequence,ID_version_seq))   

except:
        print("Erro Povoação Sequences")

In [234]:
# cur.execute("""
#                 INSERT INTO Sequences ( Count_A, Count_C, Count_T, Count_G,Sequence, ID_version_seq)    
#                 VALUES (%s, %s, %s, %s, %s,%s)
#                 """,
#                         ( Count_A, Count_C, Count_T, Count_G, Sequence,ID_version_seq))   

# Tabela Features

In [235]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [236]:
print(ID)
print(CDS_count)
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

NC_069360
88
133
0
0
0
0
[0:153821](+)


In [237]:
try:
        cur.execute("""
        INSERT INTO Features (ID_version_genbank, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID_version_seq, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

In [238]:
# cur.execute("""
#         INSERT INTO Features (ID_version_genbank, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """,
#                 (ID_version_seq, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 

# Tabela CDS

In [239]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [240]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: [347:1409](-)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:77434723']
    Key: gene, Value: ['psbA']
    Key: locus_tag, Value: ['PEX38_pgp088']
    Key: product, Value: ['PSII 32 KDa protein']
    Key: protein_id, Value: ['YP_010591274.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MTAILERRESESLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATSVFIIAFIAAPPVDIDGIREPVSGSLLYGNNIISGAIIPTSAAIGLHFYPIWEAASVDEWLYNGGPYELIVLHFLLGVACYMGREWELSFRLGMRPWIAVAYSAPVAAATAVFLIYPIGQGSFSDGMPLGISGTFNFMIVFQAEHNILMHPFHMLGVAGVFGGSLFSAMHGSLVTSSLIRETTENESANEGYRFGQEEETYNIVAAHGYFGRLIFQYASFNNSRSLHFFLAAWPVVGIWFTALGISTMAFNLNGFNFNQSVVDSQGRVINTWADIINRANLGMEVMHERNAHNFPLDLAAVEAPSTNG']


['MTAILERRESESLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATSVFIIAFIAAPPVDIDGIREPVSGSLLYGNNIISGAIIPTSAAIGLHFYPIWEAASVDEWLYNGGPYELIVLHFLLGVACYMGREWELSFRLGMRPWIAVAYSAPVAAATAVFLIYPIGQGSFSDGMPLGISGTFNFMIVFQAEHNILMHPFHMLGVAGVFGGSLFSAMHGSLVTSSLIRETTENESANEGYRFGQEEETYNIVAAHGYFGRLIFQYASFNNSRSLHFFL

In [241]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (ID_protein_cds, Translacion_seq, Localization, Protein,id_version_features)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                (str(ID_protein[num_list]), str(Translacion_seq[num_list]), str(Localization[num_list]), str(Protein[num_list]), ID_version_seq))
        num_list += 1

# Tabela References

In [242]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [243]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("None")

    
    n = re.search(r"title:\s+([\w\s')(-]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("None")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("None")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("None") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

In [244]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)

NC_069360
['None', 'None', 'None']
[['Zuo,S', 'Guo,X', 'Mandakova,T', 'Edginton,M', 'Al-Shehbaz,I.A.', 'Lysak,M.A.'], [], ['Zuo,S.']]
['Genome diploidization associates with cladogenesis', 'Direct Submission', 'Direct Submission']
['Plant Physiol 190 (1), 403-420 (2022)', 'Submitted (10-JAN-2023) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA', 'Submitted (03-NOV-2021) CEITEC, Masaryk University, Kamenice 5, Brno 62500, Czech Republic']
['None', 'NCBI Genome Project', 'None']
['None', 'None', 'None']


In [245]:
reference_count = 0                 
for c in Pubmed:    
        cur.execute("""
                INSERT INTO reference (Journal_ID, Pubmed, Consortium, Remark,Title)
                VALUES (%s, %s, %s, %s, %s)
                """,
                (str(Journal[reference_count]), str(Pubmed[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count]), str(Title[reference_count])))
        reference_count += 1

# Tabela Authors


In [246]:
trash_list = [str(a) for c in Authors for a in c]
c = 0
while len(trash_list) > c:
    cur.execute(f"""
        INSERT INTO Authors (Names)    
        VALUES (%s)
        """,
        (trash_list[c],))
    c += 1

# Tabela Pubmed_Info

In [247]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "None":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])
        
        if re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q):
            o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
            Abstract.append(o.group(1))
        else:
            Abstract.append("None")
        
        if re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q):
            p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
            DOI.append(p.group(1))
        else:
            DOI.append("None")

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")
    if c == "None":
            Abstract.append("None")
            DOI.append("None")
            Affiliation.append("None")



In [248]:
print(Pubmed)
print(Title)
print(Abstract)
print(DOI)


['None', 'None', 'None']
['Genome diploidization associates with cladogenesis', 'Direct Submission', 'Direct Submission']
['None', 'None', 'None']
['None', 'None', 'None']


In [249]:
# try:
#         pubmed_count= 0
#         for c in Pubmed:
#                 cur.execute("""
#                         INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI)   
#                         VALUES (%s, %s, %s, %s)
#                         """,
#                         (Pubmed[pubmed_count] + f"_{pubmed_count}", Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count]))   
#         if c != "None":
#                 pubmed_count += 1
# except:
#         print("Erro Povoar Pubmed")

In [250]:
pubmed_count= 0
for c in Pubmed :
        if c != "None":
                cur.execute("""
                        INSERT INTO Pubmed_Info (ID_journal, Title, Abstract, DOI, Affiliation)   
                        VALUES (%s, %s, %s, %s, %s)
                        """,
                        (Journal[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count], Affiliation[pubmed_count]))  
                pubmed_count += 1 
        else:
                pubmed_count += 1